In [3]:
import sys
import json
sys.path.append('../../..')
from grammar.db_tool import DBTool
from grammar.llm import gpt4_llm

connection_string = 'sqlite:///../rel_database/company_employee.sqlite'
db_tool = DBTool(connection_string)
schema = db_tool.stringify_schema()
print(schema)

Table: company
	 Column: Company_ID (REAL)
	 Column: Name (TEXT)
	 Column: Headquarters (TEXT)
	 Column: Industry (TEXT)
	 Column: Sales_in_Billion (REAL)
	 Column: Profits_in_Billion (REAL)
	 Column: Assets_in_Billion (REAL)
	 Column: Market_Value_in_Billion (REAL)
Table: people
	 Column: People_ID (INTEGER)
	 Column: Age (INTEGER)
	 Column: Name (TEXT)
	 Column: Nationality (TEXT)
	 Column: Graduation_College (TEXT)
Table: employment
	 Column: Company_ID (INTEGER)
	 Column: People_ID (INTEGER)
	 Column: Year_working (INTEGER)



## Generate Template-based Profile

In [4]:
# Company Profile
companies = db_tool.query('SELECT * from Company')
company_profiles: list = []
employee_profiles: list[list] = []
for company_id, company_name, headquarter, industry, sales, profits, assets, market_value in eval(companies):
    # print(f'Company ID: {company_id}')
    company_profile = f"""
    **Name**:{company_name}
    **Headquarter**: {headquarter}
    **Industry**: {industry}
    **Sales in Billion**: {sales}
    **Profit in Billion**: {profits}
    **Asset in Billion**: {assets}
    **Market Value in Billion**: {market_value}"""
    company_profiles.append('Company Profile'+company_profile)
    
    
    # Employee Profile in the company
    # employees = db_tool.query(f'SELECT * \
    #     FROM Employment e JOIN People p ON e.People_ID = p.People_ID \
    #     WHERE e.Company_ID = {str(int(company_id))};')
    employees = db_tool.query(f"SELECT p.People_ID, p.Name, p.Age, p.Nationality, p.Graduation_College, e.Year_working \
        FROM company c JOIN employment e ON c.Company_ID = e.Company_ID \
        JOIN people p ON e.People_ID = p.People_ID \
        WHERE c.Name = '{company_name}';")
    
    employee_profiles_one = []
    if employees == '' or employees is None:
        pass
        # print(f'{company_name}: No employee found')
    else:
        
        # print(f'{company_name}: {employees}')
        
        for employee_id, age, name, nationality, graduation_college, years in eval(employees):
            employee_profile = f"""Employee Profile
    **Name**: {name}
    **Age**: {age}
    **Nationality**: {nationality}
    **Graduation College**: {graduation_college}
    **Employer**: {company_name}
    **Year working in {company_name}**: {years}"""  + '\n\n   Below is A Brief of the Employer:\n' + company_profile
            employee_profiles_one.append(employee_profile)
            
    employee_profiles.append(employee_profiles_one)

print(company_profiles[0])
print(employee_profiles[6][0])

Company Profile
    **Name**:EnergySphere
    **Headquarter**: USA
    **Industry**: Oil and gas
    **Sales in Billion**: 433.5
    **Profit in Billion**: 41.1
    **Asset in Billion**: 331.1
    **Market Value in Billion**: 407.4
Employee Profile
    **Name**: 31
    **Age**: Darryl Kennedy
    **Nationality**: United States
    **Graduation College**: Oklahoma
    **Employer**: Phoenix Petro
    **Year working in Phoenix Petro**: 1

   Below is A Brief of the Employer:

    **Name**:Phoenix Petro
    **Headquarter**: China
    **Industry**: Oil and gas
    **Sales in Billion**: 310.1
    **Profit in Billion**: 20.6
    **Asset in Billion**: 304.7
    **Market Value in Billion**: 294.7


## Generate Profile Documents

In [5]:
# Company Profiles
company_profiles_docs = {}
for i in range(len(company_profiles)):

    prompt = f"""Please create a Wikipedia-style page for a company with the following profile. 
    The generated text should naturally integrate the provided details throughout the content rather than listing them in bullet points.
    The description should be between 200 to 400 words. 
    \n{company_profiles[i]}"""
    document = gpt4_llm(prompt)
    company_profiles_docs[str(i+1)] = document

# save company profiles documents
save_path = 'company_documents.json'
with open(save_path, 'w') as f:
    json.dump(company_profiles_docs, f)
# read company profiles documents
with open(save_path, 'r') as f:
    company_profiles_docs = json.load(f)



In [32]:
employee_profiles_docs = {str(i+1): [] for i in range(len(company_profiles))}
print(employee_profiles[6][0])
for i in range(len(company_profiles)):
    print(f'Company {i+1}')
    for j in range(len(employee_profiles[i])):
        print(f'Employee {j+1}')
        prompt = f"""Please create a resume for an employee with the following information. 
        The generated text should naturally integrate all the provided facts and details throughout the content rather than listing them in bullet points.
        The description should be between 200 to 400 words. 
        \n{employee_profiles[i][j]}"""
        document = gpt4_llm(prompt)
        employee_profiles_docs[str(i+1)].append(document)

Employee Profile
    **Name**: 31
    **Age**: Darryl Kennedy
    **Nationality**: United States
    **Graduation College**: Oklahoma
    **Employer**: PetroChina
    **Year working in PetroChina**: 1

   Below is A Brief of the Employer:

    **Name**:PetroChina
    **Headquarter**: China
    **Industry**: Oil and gas
    **Sales in Billion**: 310.1
    **Profit in Billion**: 20.6
    **Asset in Billion**: 304.7
    **Market Value in Billion**: 294.7
Company 1
Company 2
Company 3
Company 4
Company 5
Company 6
Company 7
Employee 1
Company 8
Company 9
Company 10
Company 11
Employee 1
Company 12
Company 13
Employee 1
Company 14
Company 15
Employee 1
Company 16
Company 17
Employee 1
Company 18
Company 19


In [33]:
# save employee profiles documents
save_path = '../spider-database/employee_documents.json'
with open(save_path, 'w') as f:
    json.dump(employee_profiles_docs, f)
# read employee profiles documents
with open(save_path, 'r') as f:
    employee_profiles_docs = json.load(f)